In [1]:
import openai
from df_utils import *
import re
import random
import numpy as np
import torch
# from utils import *
openai.api_key = open("key.txt", "r").read().strip('\n')

In [2]:
EPISODE_ITEMS = 32
WALL_ITEMS = EPISODE_ITEMS / 2

In [3]:
def wall_evaluator_gpt4(num_default=0, wall_skip=0):
    df_new = DataFrameUtils().preprocess_text()
    wall_info = {}
    num = (num_default * EPISODE_ITEMS) + (wall_skip * WALL_ITEMS)
    wall1_default = df_new[int(num - WALL_ITEMS):int(num)]['Names'].tolist()
    wall_info['names'] = wall1_default
    wall1_shuffled = wall1_default.copy()
    random.shuffle(wall1_shuffled)
    wall1 = " ".join(wall1_shuffled)
    return wall1, wall1_default

In [4]:
def get_completion(model, num_default=0, wall_skip=0):
    completion = openai.ChatCompletion.create(
    model="gpt-4",
    # max_tokens=125,
    messages=[{"role": "system", "content": "You are a helpful assistant that solves only connect walls."},
              {"role": "assistant", "content": "for each group describe the relationship in first line and group words in second single line"},
              {"role": "user", "content": "group the following words into 4 groups of 4 : " + wall_evaluator_gpt4(num_default, wall_skip)[0]}],
                 temperature=0)
    return completion.choices[0].message.content

In [5]:
def examine_wall(gpt_content, num_default=0, wall_skip=0):
    GROUP_WORDS = 4
    ITEM_NUM = 16
    reply_content = gpt_content
    lst_words = []
    lst_groups = []
    flag = False
    if '-' in reply_content:
        flag = True
        reply_content = reply_content.replace('- ', '')
    reply = reply_content.split('\n\n')
    for i in range(4):
        replay_split = reply[i].split('\n')
        if flag:
            lst_words += replay_split[1:]
        else:
            lst_words += replay_split[1].split(', ')
        lst_groups += re.findall(r'(?<=: )(.*)', replay_split[0])*4

    lst_words_default = wall_evaluator_gpt4(num_default, wall_skip)[1]
    correct_groups = 0
    for i in range(GROUP_WORDS, ITEM_NUM+1, GROUP_WORDS):
        for j in range(GROUP_WORDS, ITEM_NUM+1, GROUP_WORDS):
            if sorted(lst_words[i-GROUP_WORDS:i]) == sorted(lst_words_default[j-GROUP_WORDS:j]):
                correct_groups += 1
                print(lst_words[i-GROUP_WORDS:i])
    return correct_groups, lst_words, lst_groups

In [6]:
wall_evaluator_gpt4(num_default=0, wall_skip=1)

('grass snout santa screw up strange bottom down satan elastic-band dr-riviera charm squash-ball condom lag puck',
 ['elastic-band',
  'puck',
  'squash-ball',
  'condom',
  'screw',
  'grass',
  'snout',
  'lag',
  'santa',
  'satan',
  'bottom',
  'dr-riviera',
  'up',
  'down',
  'charm',
  'strange'])

In [12]:
# test gpt-4 performance
solved_wall = get_completion('gpt-4', num_default=0, wall_skip=3)
correct_groups, lst_words, lst_groups = examine_wall(solved_wall, num_default=0, wall_skip=3)
print('number of correct groups: ', correct_groups)
print('------------------')
print('solved_words: ', lst_words)
print('------------------')
print('solved_groups: ', lst_groups)

['carter', 'bush', 'gore', 'dole']
['ash', 'willow', 'lime', 'cork']
['galway', 'kerry', 'clare', 'mayo']
['lamb', 'gower', 'may', 'fry']
number of correct groups:  4
------------------
solved_words:  ['carter', 'bush', 'gore', 'dole', 'ash', 'willow', 'lime', 'cork', 'galway', 'kerry', 'clare', 'mayo', 'lamb', 'gower', 'may', 'fry']
------------------
solved_groups:  ['Former US Presidents or Vice Presidents', 'Former US Presidents or Vice Presidents', 'Former US Presidents or Vice Presidents', 'Former US Presidents or Vice Presidents', 'Types of trees', 'Types of trees', 'Types of trees', 'Types of trees', 'Irish counties', 'Irish counties', 'Irish counties', 'Irish counties', 'British public figures with surnames that are also verbs', 'British public figures with surnames that are also verbs', 'British public figures with surnames that are also verbs', 'British public figures with surnames that are also verbs']


In [13]:
print(solved_wall)

Group 1: Former US Presidents or Vice Presidents
carter, bush, gore, dole

Group 2: Types of trees
ash, willow, lime, cork

Group 3: Irish counties
galway, kerry, clare, mayo

Group 4: British public figures with surnames that are also verbs
lamb, gower, may, fry
